In [1]:
import os
import sys
import os.path as op
import numpy as np
from matplotlib import pyplot as plt

sys.path.append('..')
from scripts.data import DataInterface
from scripts.model import ModelInteface

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(c_count.value)? (pynvml.py, line 1671)

In [5]:
data = DataInterface(num_workers=1, dataset='mask_dataset', batch_size=1, 
                     tore_dir="/mnt/nfs/work1/trahman/zhongyangzha/dvs_hpe/ntore_dataset/synthetic", 
                     mask_dir='/mnt/nfs/work1/trahman/zhongyangzha/dvs_hpe/ntore_dataset/masks', 
                     percentile=90, seq_len=16, acc_time=0.02, step_size=0.02, 
                     cache_size=1, base_number=128, test_characters=['Miku'])
model = ModelInteface(model_name='unet', batch_size=1, use_convlstm=False, bilinear=False, loss='bce')

NameError: name 'DataInterface' is not defined

In [ ]:
# ntores, masks = next(iter(data.test_dataloader()))
# ntores = ntores.reshape((ntores.shape[0]*ntores.shape[1], *ntores.shape[2:]))
# res = model(ntores)

In [ ]:
idx = 2
batch = data.testset[idx]
ntores, masks = batch
res = model.test_step(batch, idx)